In [135]:
#importar bibliotecas
#!pip install -U scikit-learn
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression , Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , f1_score, precision_score, recall_score
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import  GridSearchCV , train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler ,PolynomialFeatures,normalize
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn import preprocessing
from pandas.plotting import autocorrelation_plot
from pandas.plotting import scatter_matrix
%matplotlib inline

In [136]:
df = pd.read_csv("dados/transformado.csv", sep=",")

In [137]:
#modificando o resultado para numerico
#df.loc[(df['resultado']=="V"),'resultado'] = 1
#df.loc[(df['resultado']=="D"),'resultado'] = 2
#df.loc[(df['resultado']=="E"),'resultado'] = 3

#df.loc[(df['mv']=="m"),'mandante'] = 1
#df.loc[(df['mv']=="v"),'mandante'] = 0
#df.loc[(df['mv']=="v"),'visitante'] = 1
#df.loc[(df['mv']=="m"),'visitante'] = 0

#df['mandante'] = df['mandante'].astype(int)
#df['visitante'] = df['visitante'].astype(int)
#df['resultado'] = df['resultado'].astype(int)

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1876 entries, 0 to 1875
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   partida_id                 1876 non-null   int64  
 1   rodada                     1876 non-null   int64  
 2   mandante                   1876 non-null   object 
 3   visitante                  1876 non-null   object 
 4   formacao_mandante          1876 non-null   object 
 5   formacao_visitante         1876 non-null   object 
 6   vencedor                   1876 non-null   object 
 7   mandante_placar            1876 non-null   int64  
 8   visitante_placar           1876 non-null   int64  
 9   clube_mandante             1876 non-null   object 
 10  chutes_mandante            1876 non-null   int64  
 11  chutes_no_alvo_mandante    1876 non-null   int64  
 12  posse_de_bola_mandante     1876 non-null   float64
 13  passes_mandante            1876 non-null   int64

In [139]:
df.tail()

,partida_id,rodada,mandante,visitante,formacao_mandante,formacao_visitante,vencedor,mandante_placar,visitante_placar,clube_mandante,...,passes_visitante,precisao_passes_visitante,faltas_visitante,cartao_amarelo_visitante,cartao_vermelho_visitante,impedimentos_visitante,escanteios_visitante,temporada,resultado,total_gols
1871,8021,38,Cuiaba,Coritiba,4-1-4-1,4-1-4-1,Cuiaba,2,1,Cuiaba,...,389,0.82,8,3,1,2,1,2022,1,3
1872,8022,38,Bragantino,Fluminense,4-2-3-1,4-2-3-1,Fluminense,0,1,Bragantino,...,463,0.82,15,2,0,0,9,2022,2,1
1873,8023,38,Corinthians,Atletico-MG,4-1-4-1,4-2-3-1,Atletico-MG,0,1,Corinthians,...,351,0.82,19,6,0,0,4,2022,2,1
1874,8024,38,Internacional,Palmeiras,4-2-3-1,4-2-3-1,Internacional,3,0,Internacional,...,306,0.82,17,3,0,2,5,2022,1,3
1875,8025,38,Goias,Sao Paulo,4-2-3-1,4-1-2-1-2,Sao Paulo,0,4,Goias,...,591,0.90,14,1,0,0,4,2022,2,4


In [140]:
# listando features categoricas e numéricas:
categorical_attributes = list(df.select_dtypes(include=['object']).columns)
numerical_attributes = list(df.select_dtypes(include=['float64', 'int64']).columns)
print('categorical_attributes:', categorical_attributes)
print('numerical_attributes:', numerical_attributes)

categorical_attributes: ['mandante', 'visitante', 'formacao_mandante', 'formacao_visitante', 'vencedor', 'clube_mandante', 'clube_visitante']
numerical_attributes: ['partida_id', 'rodada', 'mandante_placar', 'visitante_placar', 'chutes_mandante', 'chutes_no_alvo_mandante', 'posse_de_bola_mandante', 'passes_mandante', 'precisao_passes_mandante', 'faltas_mandante', 'cartao_amarelo_mandante', 'cartao_vermelho_mandante', 'impedimentos_mandante', 'escanteios_mandante', 'rodata_visitante', 'chutes_visitante', 'chutes_no_alvo_visitante', 'posse_de_bola_visitante', 'passes_visitante', 'precisao_passes_visitante', 'faltas_visitante', 'cartao_amarelo_visitante', 'cartao_vermelho_visitante', 'impedimentos_visitante', 'escanteios_visitante', 'temporada', 'resultado', 'total_gols']


In [141]:
#Deixar somente as variáveis numericas 
num_data  =df.drop(['partida_id','rodada','mandante', 'visitante', 'total_gols','vencedor'],axis=1)
num_data

,formacao_mandante,formacao_visitante,mandante_placar,visitante_placar,clube_mandante,chutes_mandante,chutes_no_alvo_mandante,posse_de_bola_mandante,passes_mandante,precisao_passes_mandante,...,posse_de_bola_visitante,passes_visitante,precisao_passes_visitante,faltas_visitante,cartao_amarelo_visitante,cartao_vermelho_visitante,impedimentos_visitante,escanteios_visitante,temporada,resultado
0,4-2-3-1,4-2-3-1,0,1,Cruzeiro,12,2,0.40,425,0.81,...,0.60,649,0.86,15,1,1,3,4,2018,2
1,4-2-3-1,4-1-4-1,2,2,Vitoria,21,6,0.66,545,0.88,...,0.34,286,0.72,10,2,1,2,5,2018,3
2,4-3-3,4-1-4-1,2,0,Santos,22,3,0.54,532,0.90,...,0.46,472,0.85,5,1,0,0,2,2018,1
3,4-1-4-1,4-3-2-1,3,0,America-MG,14,4,0.38,363,0.76,...,0.62,580,0.85,12,3,0,3,6,2018,1
4,4-1-4-1,4-1-4-1,2,1,Vasco,19,7,0.67,596,0.88,...,0.33,321,0.78,18,3,0,3,3,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,4-1-4-1,4-1-4-1,2,1,Cuiaba,17,8,0.57,505,0.88,...,0.43,389,0.82,8,3,1,2,1,2022,1
1872,4-2-3-1,4-2-3-1,0,1,Bragantino,13,4,0.41,319,0.77,...,0.59,463,0.82,15,2,0,0,9,2022,2
1873,4-1-4-1,4-2-3-1,0,1,Corinthians,17,4,0.59,494,0.84,...,0.41,351,0.82,19,6,0,0,4,2022,2
1874,4-2-3-1,4-2-3-1,3,0,Internacional,15,8,0.48,287,0.77,...,0.52,306,0.82,17,3,0,2,5,2022,1


In [142]:
#separa as features 
features = num_data.drop(['resultado'],axis=1)
#separa as labels
labels = num_data['resultado']


print('Features')
print (features.head())

print (" ")
print ('#############')
print (" ")


print ('Labels')
print (labels.head()) 

Features
  formacao_mandante formacao_visitante  mandante_placar  visitante_placar  \
0           4-2-3-1            4-2-3-1                0                 1   
1           4-2-3-1            4-1-4-1                2                 2   
2             4-3-3            4-1-4-1                2                 0   
3           4-1-4-1            4-3-2-1                3                 0   
4           4-1-4-1            4-1-4-1                2                 1   

  clube_mandante  chutes_mandante  chutes_no_alvo_mandante  \
0       Cruzeiro               12                        2   
1        Vitoria               21                        6   
2         Santos               22                        3   
3     America-MG               14                        4   
4          Vasco               19                        7   

   posse_de_bola_mandante  passes_mandante  precisao_passes_mandante  ...  \
0                    0.40              425                      0.81  ...   
1

In [143]:
#transformando as formações em dados numericos
le = preprocessing.LabelEncoder()
le.fit(features['formacao_mandante'])
features['formacao_mandante'] = le.transform(features["formacao_mandante"])
features['formacao_visitante'] = le.transform(features["formacao_visitante"])

print('Features')
print (features.head())

Features
   formacao_mandante  formacao_visitante  mandante_placar  visitante_placar  \
0                 11                  11                0                 1   
1                 11                   9                2                 2   
2                 15                   9                2                 0   
3                  9                  14                3                 0   
4                  9                   9                2                 1   

  clube_mandante  chutes_mandante  chutes_no_alvo_mandante  \
0       Cruzeiro               12                        2   
1        Vitoria               21                        6   
2         Santos               22                        3   
3     America-MG               14                        4   
4          Vasco               19                        7   

   posse_de_bola_mandante  passes_mandante  precisao_passes_mandante  ...  \
0                    0.40              425                      0.

In [144]:
le = preprocessing.LabelEncoder()
le.fit(features['clube_mandante'])
features['clube_mandante'] = le.transform(features["clube_mandante"])
features['clube_visitante'] = le.transform(features["clube_visitante"])

print('Features')
print (features.head())

Features
   formacao_mandante  formacao_visitante  mandante_placar  visitante_placar  \
0                 11                  11                0                 1   
1                 11                   9                2                 2   
2                 15                   9                2                 0   
3                  9                  14                3                 0   
4                  9                   9                2                 1   

   clube_mandante  chutes_mandante  chutes_no_alvo_mandante  \
0              13               12                        2   
1              28               21                        6   
2              24               22                        3   
3               0               14                        4   
4              27               19                        7   

   posse_de_bola_mandante  passes_mandante  precisao_passes_mandante  ...  \
0                    0.40              425                  

In [145]:
features.isna().sum()

formacao_mandante            0
formacao_visitante           0
mandante_placar              0
visitante_placar             0
clube_mandante               0
chutes_mandante              0
chutes_no_alvo_mandante      0
posse_de_bola_mandante       0
passes_mandante              0
precisao_passes_mandante     0
faltas_mandante              0
cartao_amarelo_mandante      0
cartao_vermelho_mandante     0
impedimentos_mandante        0
escanteios_mandante          0
rodata_visitante             0
clube_visitante              0
chutes_visitante             0
chutes_no_alvo_visitante     0
posse_de_bola_visitante      0
passes_visitante             0
precisao_passes_visitante    0
faltas_visitante             0
cartao_amarelo_visitante     0
cartao_vermelho_visitante    0
impedimentos_visitante       0
escanteios_visitante         0
temporada                    0
dtype: int64

In [146]:
#Escoolhendo as melhores features com Kbest

features_list = ('formacao_mandante','formacao_visitante','mandante_placar','visitante_placar',
                 'chutes_mandante','chutes_no_alvo_mandante','posse_de_bola_mandante',
                 'passes_mandante','precisao_passes_mandante','faltas_mandante','cartao_amarelo_mandante',
                 'cartao_vermelho_mandante','impedimentos_mandante','escanteios_mandante',
                 'rodata_visitante','chutes_visitante','chutes_no_alvo_visitante','posse_de_bola_visitante',
                 'passes_visitante','precisao_passes_visitante','faltas_visitante','cartao_amarelo_visitante',
                 'cartao_vermelho_visitante','impedimentos_visitante','escanteios_visitante')


k_best_features = SelectKBest(k='all')
k_best_features.fit_transform(features, labels)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:27])
best_features = k_best_features_final.keys()
print ('')
print ("Melhores features:")
print (k_best_features_final)


Melhores features:
{'visitante_placar': 639.0828917349123, 'chutes_mandante': 586.1448988782514, 'precisao_passes_visitante': 109.92280025097195, 'passes_mandante': 95.00931987283516, 'cartao_vermelho_visitante': 25.275046892805076, 'escanteios_mandante': 24.118624856532527, 'faltas_visitante': 13.297094431728825, 'precisao_passes_mandante': 13.29709443172865, 'passes_visitante': 8.086673146878562, 'cartao_amarelo_visitante': 7.454636275026087, 'cartao_vermelho_mandante': 7.260795762587245, 'impedimentos_visitante': 6.128429360040805, 'chutes_visitante': 4.836083904268544, 'faltas_mandante': 4.143359126060914, 'cartao_amarelo_mandante': 3.7560772607207937, 'posse_de_bola_visitante': 2.695897849905467, 'chutes_no_alvo_visitante': 2.1501796742460497, 'rodata_visitante': 2.093399331828941, 'impedimentos_mandante': 2.0199450096007197, 'chutes_no_alvo_mandante': 1.552505097992794, 'posse_de_bola_mandante': 1.1718419229248394, 'formacao_visitante': 0.9017953040211832, 'mandante_placar': 0.8

In [147]:
#separa as features com base no Kbest
#oldfeatures  = num_data.drop(['cartao_amarelo_mandante', 'impedimentos_visitante', 'cartao_amarelo_visitante', 'temporada', 'rodata_visitante', 'precisao_passes_mandante', 'faltas_mandante', 'chutes_visitante', 'escanteios_mandante', 'cartao_vermelho_mandante', 'escanteios_visitante', 'chutes_no_alvo_mandante', 'cartao_vermelho_visitante'],axis=1)
features  = features.drop(['cartao_vermelho_visitante','escanteios_mandante','faltas_visitante', 'precisao_passes_mandante', 'passes_visitante', 'cartao_amarelo_visitante', 'cartao_vermelho_mandante', 'impedimentos_visitante', 'chutes_visitante', 'faltas_mandante','cartao_amarelo_mandante', 'posse_de_bola_visitante', 'chutes_no_alvo_visitante', 'rodata_visitante', 'impedimentos_mandante', 'chutes_no_alvo_mandante', 'posse_de_bola_mandante', 'formacao_visitante', 'mandante_placar', 'escanteios_visitante'],axis=1)

print('Features')
print (features.head())

print ('=========')

print ('Labels')
print (Y.head())

Features
   formacao_mandante  visitante_placar  clube_mandante  chutes_mandante  \
0                 11                 1              13               12   
1                 11                 2              28               21   
2                 15                 0              24               22   
3                  9                 0               0               14   
4                  9                 1              27               19   

   passes_mandante  clube_visitante  precisao_passes_visitante  temporada  
0              425               19                       0.86       2018  
1              545               15                       0.72       2018  
2              532                9                       0.85       2018  
3              363               26                       0.85       2018  
4              596                3                       0.78       2018  
Labels
0    2
1    3
2    1
3    1
4    1
Name: resultado, dtype: int64


In [148]:
#devido ao OVERFITING, estou realizando um Low Variance Filter como tentativa de resolução
#normalize = normalize(features) 
#data_scaled = pd.DataFrame(normalize)
#data_scaled.var()

In [149]:
#storing the variance and name of variables
#variance = data_scaled.var()
#columns = features.columns

#print(variance)

In [150]:
#saving the names of variables having variance more than a threshold value

#variable = [ ]

#or i in range(0,len(variance)):
#    if variance[i]>=0.0: #setting the threshold as 1%
#        variable.append(columns[i-1])


#variable

#for i in range(0,len(variance)):
#    print(variance[i])

In [151]:
# criando um novo dataframe usando as variáveis ​​acima
#new_data = features[variable]

 # primeiras cinco linhas dos novos dados
#new_data.head() 

In [152]:
# Normalizando os dados de entrada(features)
scaler = MinMaxScaler().fit(features)
X = scaler.transform(features)

print ('Features: ',X.shape)
print (X)

Features:  (1876, 8)
[[0.55       0.2        0.46428571 ... 0.67857143 0.88679245 0.        ]
 [0.55       0.4        1.         ... 0.53571429 0.62264151 0.        ]
 [0.75       0.         0.85714286 ... 0.32142857 0.86792453 0.        ]
 ...
 [0.45       0.2        0.39285714 ... 0.10714286 0.81132075 1.        ]
 [0.55       0.         0.71428571 ... 0.78571429 0.81132075 1.        ]
 [0.55       0.8        0.64285714 ... 0.89285714 0.96226415 1.        ]]


In [153]:
# Realiza a divisão dos dados de treine e teste e deixa do espaço 2650 a 3090 para validação do ml
#X_train = features_scale[:2000 ]
#X_test = features_scale[2000:2650]
#y_train = labels[:2000]
#y_test = labels[2000:2650]

# Realiza a divisão de dados de treino e teste, separando 20% para teste

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

print( len(X_train), len(y_train))

print( len(X_test), len(y_test))

print(Y.shape)

1500 1500
376 376
(1876,)


In [154]:
#Treinando e testando os modelos
print ('LogisticRegression')

clf_LR = LogisticRegression(multi_class='multinomial',max_iter=2000)
clf_LR.fit(X_train, y_train)
pred= clf_LR.predict(X_train)

lg_acc = accuracy_score(y_train, pred)
f1=f1_score(y_train,pred,average = 'micro')
print ('Acurácia LogisticRegression:{}'.format(lg_acc))
print ('F1 Score:{}'.format(f1) )

LogisticRegression
Acurácia LogisticRegression:0.6146666666666667
F1 Score:0.6146666666666667


In [155]:
#Testando LogistRegression hyper parameters

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid)

search.fit(X_train,y_train)
clf = search.best_estimator_
pred= clf.predict(X_train)
lg_acc = accuracy_score(y_train, pred)


f1=f1_score(y_train,pred,average = 'macro')

print ('Acurácia LogisticRegression:{}'.format(lg_acc))
print ('F1 Score:{}'.format(f1) )

print (clf)

Acurácia LogisticRegression:0.6153333333333333
F1 Score:0.5248008189250762
LogisticRegression(C=10, max_iter=1000)


In [156]:
#Treinando e testando os modelos
print ('SVC')


clf = SVC()
clf.fit(X_train, y_train)
pred= clf.predict(X_train)

svc_acc = accuracy_score(y_train, pred)
f1=f1_score(y_train,pred, average='micro')
print ('Acurácia SVC:{}'.format(svc_acc))
print ('F1 Score:{}'.format(f1) )

SVC
Acurácia SVC:0.6393333333333333
F1 Score:0.6393333333333333


In [157]:
#Testando SVC hyper parameters

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

search = GridSearchCV(SVC(), param_grid)

search.fit(X_train,y_train)
clf_SVC = search.best_estimator_
pred= clf_SVC.predict(X_train)
acc = accuracy_score(y_train, pred)


f1=f1_score(y_train,pred,average = 'micro')

print ('F1 Score:{}'.format(f1))

print ('Acurácia LogisticRegression:{}'.format(acc))

print(clf_SVC)

F1 Score:0.6393333333333333
Acurácia LogisticRegression:0.6393333333333333
SVC(C=1)


In [158]:
#Treinando e testando os modelos
print ('Decision Tree')


clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
pred= clf.predict(X_train)

dt_acc = accuracy_score(y_train, pred)
f1=f1_score(y_train,pred, average='macro')
print ('Acurácia Tree:{}'.format(dt_acc))
print ('F1 Score:{}'.format(f1) )


n_estimators = [10, 50, 100, 200]
max_depth = [3, 10, 20, 40]

Decision Tree
Acurácia Tree:1.0
F1 Score:1.0


In [159]:
#Treinando e testando os modelos
print ('Naive baeys')


clf = GaussianNB()
clf.fit(X_train, y_train)
pred= clf.predict(X_train)

nb_acc = accuracy_score(y_train, pred)
f1=f1_score(y_train,pred, average='micro')
print ('Acurácia Naive baeys:{}'.format(nb_acc))
print ('F1 Score:{}'.format(f1) )

Naive baeys
Acurácia Naive baeys:0.6146666666666667
F1 Score:0.6146666666666667


In [160]:
from sklearn.model_selection import cross_val_score
SEED = 42
np.random.seed(SEED)
model = DecisionTreeClassifier(max_depth=3)
results = cross_val_score(model, X_train, 
                          y_train, cv = 5, scoring = 'accuracy')
def intervalo(results):
    mean = results.mean()
    dv = results.std()
    print('Acurácia média: {:.2f}%'.format(mean*100))
    print('Intervalo de acurácia: [{:.2f}% ~ {:.2f}%]'.format((mean - 2*dv)*100, (mean + 2*dv)*100))
    
intervalo(results)

Acurácia média: 59.93%
Intervalo de acurácia: [53.46% ~ 66.40%]


In [161]:
#Executando a previsao

previsao=features_scale[2650:]

game_id_full=df['partida_id']
game_id=game_id_full[2650:]

res_full=df['resultado']
res=res_full[2650:]


pred=clf_LR.predict(previsao)

df=pd.DataFrame({'real': res, 'previsao':pred, 'partida_id':game_id})

print(df)

NameError: name 'features_scale' is not defined

In [ ]:
#confusion Matrix

df=pd.DataFrame(df,columns=['real','previsao' ])

cf_matrix=pd.crosstab(df['real'], df['previsao'], rownames=['real'] , colnames=['previsao'])

sns.heatmap(cf_matrix, annot=True, cmap='Reds')

In [ ]:
# relatório do modelo
print('Relatório de classificação:\n', classification_report(y_train, y_pred, digits=4))

In [ ]:
# Functions for plotting
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()

In [ ]:
# Let's examine the distribution of the predicted values of the training data
Title = '-'
DistributionPlot(y_train, yhat_train, "Actual Values (Train)", "Predicted Values (Train)", Title)

In [ ]:


# Let's examine the distribution of the predicted values of the training data
Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_test, yhat_teste, "Actual Values (teste)", "Predicted Values (teste)", Title)
